# The Code for Data Integration

This .ipynb describes how we constructed the master state-month feature dataset used in the AirHealth project. The goal is to integrate multiple air pollutants, meteorolgical, urban, health outcome sources into a single, model-building-ready table indexed by U.S. state and calendar month.

### Step 1: State-Month Grid (Skeleton)

In [ ]:
import pandas as pd

# The 50 States
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut",
    "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan",
    "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina",
    "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
    "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

# Set time period 2018-01 ~ 2025-07
months = pd.period_range("2018-01", "2025-07", freq="M")

# state × month: Construct the raw grid for data skeleton
grid = (
    pd.MultiIndex.from_product([states, months], names=["state", "month_period"])
      .to_frame(index=False)
)

grid["month"] = grid["month_period"].astype(str)  # "YYYY-MM"

# period
grid = grid.drop(columns=["month_period"])

# ============================
# 1) Adding feature columns
# ============================

#  VMT, NDVI, Flights
basic_cols = ["vmt", "ndvi", "flights"]

# Air pollutants
pollutants = ["co", "no2", "pm25", "pm10", "so2", "o3"]
stats = ["mean", "max1_value", "max1_hour", "aqi"]

pollutant_cols = [f"{p}_{s}" for p in pollutants for s in stats]

# Meteorological data
met_cols = ["awnd", "prcp", "tavg", "tmax", "tmin", "tsun"]

# All features List
feature_cols = basic_cols + pollutant_cols + met_cols

# set N/A
for col in feature_cols:
    grid[col] = pd.NA

print(grid.head())
print("The number of Row:", len(grid))
print("The number of Column:", len(grid.columns))


### Step 2: EPA Air Pollutants

In [ ]:
import pandas as pd
from pathlib import Path


# Define Pollutants Dataset
pollutant_files = {
    "co":    "./dataset/EQS/co_state_month_2018_2025.csv",
    "no2":   "./dataset/EQS/no2_state_month_2018_2025.csv",
    "o3":    "./dataset/EQS/ozone_state_month_2018_2025.csv",
    "pm25":  "./dataset/EQS/PM2.5_state_month_2018_2025.csv",
    "pm10":  "./dataset/EQS/PM10_Mass_state_month_2018_2025.csv",
    "so2":   "./dataset/EQS/so2_state_month_2018_2025.csv",
}

# Set index as (state, month)
grid_indexed = grid.set_index(["state", "month"])

for prefix, fname in pollutant_files.items():
    print(f"Filling values from {fname} for {prefix} ...")

    df = pd.read_csv(fname)
    df["Month"] = pd.to_datetime(df["Month"]).dt.to_period("M").astype(str)

    # rename
    df = df.rename(columns={
        "State Name": "state",
        "Month": "month",
        "Arithmetic Mean": f"{prefix}_mean",
        "1st Max Value":   f"{prefix}_max1_value",
        "1st Max Hour":    f"{prefix}_max1_hour",
        "AQI":             f"{prefix}_aqi",
    })

    value_cols = [f"{prefix}_mean",
                  f"{prefix}_max1_value",
                  f"{prefix}_max1_hour",
                  f"{prefix}_aqi"]

    df_use = df[["state", "month"] + value_cols].copy()

    df_use = df_use.set_index(["state", "month"])

    # Fill Values
    common_idx = grid_indexed.index.intersection(df_use.index)
    grid_indexed.loc[common_idx, value_cols] = df_use.loc[common_idx, value_cols].values

grid_filled = grid_indexed.reset_index()

print(grid_filled.head())


###Step 3: Urban Data

In [ ]:
import pandas as pd


vmt = pd.read_csv("./dataset/Eunsu/FHWA_VMT_All_new.csv")


vmt["year"] = vmt["year"].astype(int)
vmt["month"] = vmt["month"].astype(int)

vmt["month"] = pd.PeriodIndex(
    year=vmt["year"],
    month=vmt["month"],
    freq="M"
).astype(str)   # "2004-04", "2018-01"


vmt_use = vmt[(vmt["month"] >= "2018-01") & (vmt["month"] <= "2025-07")]

vmt_use = vmt_use[["state", "month", "vmt"]].copy()

if "vmt" not in grid_filled.columns:
    grid_filled["vmt"] = pd.NA


g2 = grid_filled.set_index(["state", "month"])
v2 = vmt_use.set_index(["state", "month"])


common_idx = g2.index.intersection(v2.index)
g2.loc[common_idx, "vmt"] = v2.loc[common_idx, "vmt"].values


grid_filled = g2.reset_index()
print(grid_filled.head())

print(grid_filled[grid_filled["vmt"].notna()].head())


###Step 4: Meteorological Data

In [ ]:
import pandas as pd

met = pd.read_csv("./dataset/Eunsu/GHCND_US_monthly_summary.csv")
# period_start, frequency, fips, state, AWND, PRCP, SNOW, SNWD, TAVG, TMAX, TMIN


met["month"] = pd.to_datetime(
    met["period_start"], errors="coerce"
).dt.to_period("M").astype(str)

met = met[(met["month"] >= "2018-01") & (met["month"] <= "2025-07")]

met_use = met[[
    "state", "month",
    "AWND", "PRCP", "SNOW", "SNWD", "TAVG", "TMAX", "TMIN"
]].rename(columns={
    "AWND": "awnd",
    "PRCP": "prcp",
    "SNOW": "snow",
    "SNWD": "snwd",
    "TAVG": "tavg",
    "TMAX": "tmax",
    "TMIN": "tmin",
})



for col in ["awnd", "prcp", "snow", "snwd", "tavg", "tmax", "tmin"]:
    if col not in grid_filled.columns:
        grid_filled[col] = pd.NA

g2 = grid_filled.set_index(["state", "month"])
m2 = met_use.set_index(["state", "month"])


common_idx = g2.index.intersection(m2.index)


cols = ["awnd", "prcp", "snow", "snwd", "tavg", "tmax", "tmin"]
g2.loc[common_idx, cols] = m2.loc[common_idx, cols].values


grid_filled = g2.reset_index()


print(grid_filled[grid_filled["awnd"].notna()].head())


### Step 5: Health Outcome Data

In [ ]:
import pandas as pd


df = pd.read_csv("./dataset/ihd_final.csv")

df["Deaths"] = pd.to_numeric(df["Deaths"], errors="coerce").fillna(0)


df["month"] = (
    pd.to_datetime(df["Month Code"].astype(str), errors="coerce")
      .dt.to_period("M")
      .astype(str)
)

ihd_state_month = (
    df.groupby(["Residence State", "month"], as_index=False)["Deaths"]
      .sum()
)

ihd_state_month = ihd_state_month.rename(columns={
    "Residence State": "state",
    "Deaths": "deaths"
})

print(ihd_state_month.head())


In [ ]:

if "ihd_deaths" not in grid_filled.columns:
    grid_filled["ihd_deaths"] = pd.NA

g2 = grid_filled.set_index(["state", "month"])
i2 = ihd_state_month.set_index(["state", "month"])

common_idx = g2.index.intersection(i2.index)
g2.loc[common_idx, "ihd_deaths"] = i2.loc[common_idx, "deaths"].values

grid_filled = g2.reset_index()


In [ ]:
import pandas as pd

df = pd.read_csv("./dataset/copd_final.csv")


df["Deaths"] = pd.to_numeric(df["Deaths"], errors="coerce").fillna(0)

df["month"] = (
    pd.to_datetime(df["Month Code"].astype(str), errors="coerce")
      .dt.to_period("M")
      .astype(str)
)


copd_state_month = (
    df.groupby(["Residence State", "month"], as_index=False)["Deaths"]
      .sum()
)


copd_state_month = copd_state_month.rename(columns={
    "Residence State": "state",
    "Deaths": "deaths"
})

print(copd_state_month.head())


In [ ]:

if "copd_deaths" not in grid_filled.columns:
    grid_filled["copd_deaths"] = pd.NA

g2 = grid_filled.set_index(["state", "month"])
i2 = copd_state_month.set_index(["state", "month"])

common_idx = g2.index.intersection(i2.index)
g2.loc[common_idx, "copd_deaths"] = i2.loc[common_idx, "deaths"].values

grid_filled = g2.reset_index()


In [ ]:
import pandas as pd


df = pd.read_csv("./dataset/asthma_final.csv")

df["Deaths"] = pd.to_numeric(df["Deaths"], errors="coerce").fillna(0)


df["month"] = (
    pd.to_datetime(df["Month Code"].astype(str), errors="coerce")
      .dt.to_period("M")
      .astype(str)
)


asthma_state_month = (
    df.groupby(["Residence State", "month"], as_index=False)["Deaths"]
      .sum()
)


asthma_state_month = asthma_state_month.rename(columns={
    "Residence State": "state",
    "Deaths": "deaths"
})

print(asthma_state_month.head())


In [ ]:
if "asthma_deaths" not in grid_filled.columns:
    grid_filled["asthma_deaths"] = pd.NA

g2 = grid_filled.set_index(["state", "month"])
i2 = asthma_state_month.set_index(["state", "month"])

common_idx = g2.index.intersection(i2.index)
g2.loc[common_idx, "asthma_deaths"] = i2.loc[common_idx, "deaths"].values

grid_filled = g2.reset_index()
